In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
import matplotlib.pyplot as plt
import matplotlib.animation as animation

2024-07-15 16:50:34.451445: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 16:50:34.546386: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 16:50:36.346379: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import cv2, os, glob

np.set_printoptions(threshold=np.inf) # NumPy 설정
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # CUDA 가시성 설정 (GPU 사용시 필요)

# GPU 메모리 구성 설정 (GPU 사용시 필요)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # TensorFlow 2.x에서는 set_memory_growth 대신에 메모리 할당 옵션을 설정하는 코드
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)

In [3]:
# Custom abTanh Layer
class abTanh(layers.Layer):
    def __init__(self, units=None, init_a=7, init_b=0, **kwargs):
        super(abTanh, self).__init__(**kwargs)
        self.units = units
        self.init_a = init_a
        self.init_b = init_b

    def build(self, input_shape):
        units_shape = self.units if self.units else input_shape[1:]
        self.b = self.add_weight(shape=units_shape, initializer=tf.constant_initializer(self.init_b), trainable=True, name="b")
        self.a = self.add_weight(shape=units_shape, initializer=tf.constant_initializer(self.init_a), trainable=True, name="a")

    def call(self, inputs, **kwargs):
        x = tf.subtract(inputs, self.b)
        x = tf.nn.tanh(x)
        x = tf.multiply(x, self.a)
        return x


In [4]:
# Data preparation
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
y = np.array([[0], [1], [1], [0]], dtype=np.float32)

In [5]:
# Model definition
model = models.Sequential()
model.add(layers.Input(shape=(2,)))
model.add(layers.Dense(10))
model.add(abTanh(units=10))
model.add(layers.Dense(1, activation='sigmoid'))

2024-07-15 16:50:40.040918: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38357 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:af:00.0, compute capability: 8.0


In [6]:
# Model compilation
model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Plotting function
def plot_decision_boundary(model, X, y, epoch, save_dir):
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01),
                         np.arange(y_min, y_max, 0.01))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    plt.figure()
    plt.contourf(xx, yy, Z, alpha=0.8)
    plt.scatter(X[:, 0], X[:, 1], c=y[:, 0], edgecolors='k', marker='o')
    plt.title(f'Decision Boundary at Epoch {epoch}')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.savefig(os.path.join(save_dir, f'decision_boundary_epoch_{epoch}.png'))
    plt.close()

In [8]:
# Custom training loop with plot saving
class PlotCallback(tf.keras.callbacks.Callback):
    def __init__(self, plot_interval, save_dir):
        self.plot_interval = plot_interval
        self.save_dir = save_dir
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.plot_interval == 0:
            plot_decision_boundary(self.model, X, y, epoch, self.save_dir)


In [13]:
# Directory to save plots
save_dir = 'plots_ab'
plot_callback = PlotCallback(plot_interval=1, save_dir=save_dir)

In [14]:
# Model training with visualization callback
model.fit(X, y, epochs=1000, verbose=2, callbacks=[plot_callback])

Epoch 1/1000
2813/2813 [==============================] - 7s 3ms/step
1/1 - 11s - loss: 1.7806 - accuracy: 0.7500 - 11s/epoch - 11s/step
Epoch 2/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 9s - loss: 1.7496 - accuracy: 0.7500 - 9s/epoch - 9s/step
Epoch 3/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 9s - loss: 1.7198 - accuracy: 0.7500 - 9s/epoch - 9s/step
Epoch 4/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 1.6905 - accuracy: 0.7500 - 8s/epoch - 8s/step
Epoch 5/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 9s - loss: 1.6620 - accuracy: 0.7500 - 9s/epoch - 9s/step
Epoch 6/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 9s - loss: 1.6345 - accuracy: 0.7500 - 9s/epoch - 9s/step
Epoch 7/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 1.6076 - accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 8/1000
2813/2813 [==============================] - 6

Epoch 62/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 0.8930 - accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 63/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 8s - loss: 0.8865 - accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 64/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.8797 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 65/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.8733 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 66/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.8668 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 67/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 0.8605 - accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 68/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 0.8541 - accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 69/1000
2813/2813 [==============================

1/1 - 8s - loss: 0.6443 - accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 123/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 8s - loss: 0.6422 - accuracy: 0.5000 - 8s/epoch - 8s/step
Epoch 124/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.6403 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 125/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.6382 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 126/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.6362 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 127/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.6343 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 128/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.6324 - accuracy: 0.5000 - 7s/epoch - 7s/step
Epoch 129/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.6306 - accuracy: 0.5000 - 7s/

Epoch 183/1000
2813/2813 [==============================] - 7s 3ms/step
1/1 - 9s - loss: 0.5694 - accuracy: 1.0000 - 9s/epoch - 9s/step
Epoch 184/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 9s - loss: 0.5687 - accuracy: 1.0000 - 9s/epoch - 9s/step
Epoch 185/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 10s - loss: 0.5680 - accuracy: 1.0000 - 10s/epoch - 10s/step
Epoch 186/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 0.5673 - accuracy: 1.0000 - 8s/epoch - 8s/step
Epoch 187/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 0.5666 - accuracy: 1.0000 - 8s/epoch - 8s/step
Epoch 188/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 8s - loss: 0.5659 - accuracy: 1.0000 - 8s/epoch - 8s/step
Epoch 189/1000
2813/2813 [==============================] - 7s 2ms/step
1/1 - 9s - loss: 0.5653 - accuracy: 1.0000 - 9s/epoch - 9s/step
Epoch 190/1000
2813/2813 [===================

2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4969 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 304/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4963 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 305/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4956 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 306/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4950 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 307/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4943 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 308/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4937 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 309/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4931 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 310/1000
2813/2813 [==============================] - 6s 

2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4146 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 424/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4138 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 425/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4130 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 426/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4123 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 427/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4116 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 428/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4109 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 429/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.4102 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 430/1000
2813/2813 [==============================] - 6s 

2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.3229 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 544/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.3221 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 545/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.3214 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 546/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.3206 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 547/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.3199 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 548/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.3191 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 549/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.3183 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 550/1000
2813/2813 [==============================] - 6s 

2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.2344 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 664/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.2336 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 665/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.2329 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 666/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.2323 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 667/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.2316 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 668/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.2310 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 669/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.2303 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 670/1000
2813/2813 [==============================] - 6s 

2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1614 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 784/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1608 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 785/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1603 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 786/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1598 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 787/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1593 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 788/1000
2813/2813 [==============================] - 8s 3ms/step
1/1 - 10s - loss: 0.1587 - accuracy: 1.0000 - 10s/epoch - 10s/step
Epoch 789/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1582 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 790/1000
2813/2813 [==============================] - 

2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1083 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 904/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1080 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 905/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1076 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 906/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1073 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 907/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1069 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 908/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 7s - loss: 0.1065 - accuracy: 1.0000 - 7s/epoch - 7s/step
Epoch 909/1000
2813/2813 [==============================] - 6s 2ms/step
1/1 - 8s - loss: 0.1062 - accuracy: 1.0000 - 8s/epoch - 8s/step
Epoch 910/1000
2813/2813 [==============================] - 6s 

In [17]:
# Final predictions
predictions = model.predict(X)
print("Predictions:")
print(predictions)

1/1 [==============================] - 0s 47ms/step
Predictions:
[[0.06176103]
 [0.9179942 ]
 [0.9293741 ]
 [0.0860211 ]]


In [18]:
# Print model weights
for layer in model.layers:
    print(layer.name, layer.get_weights())

dense [array([[ 0.04084266,  0.5007316 ,  0.37498394, -0.45038733, -0.49450186,
         0.39346397,  0.2812678 , -0.6338972 , -0.52010626,  0.981074  ],
       [-0.49962953, -0.28099778, -0.28500447, -0.45176756,  0.9343821 ,
         0.11099385,  0.22671035,  1.0688158 , -1.049355  ,  1.0775805 ]],
      dtype=float32), array([ 1.4721118e-04,  9.1123357e-02,  1.1986318e-01,  3.8180122e-01,
        2.2020376e-01, -1.6958751e-01,  9.7699285e-02,  2.5356117e-01,
       -1.2495299e-01,  3.6601301e-03], dtype=float32)]
ab_tanh [array([-1.4721118e-04, -9.1123357e-02, -1.1986318e-01, -3.8180122e-01,
       -2.2020376e-01,  1.6958751e-01, -9.7699285e-02, -2.5356117e-01,
        1.2495299e-01, -3.6601301e-03], dtype=float32), array([7.117761 , 7.450466 , 7.3280635, 7.247841 , 7.1553674, 7.3352633,
       6.8646297, 7.270777 , 7.342079 , 7.464209 ], dtype=float32)]
dense_1 [array([[ 0.09297875],
       [-0.8055148 ],
       [-0.89532167],
       [ 0.17676617],
       [-0.8391437 ],
       [-0.

In [10]:
from PIL import Image
import os
import re


# Directory where plots are saved
save_dir = 'plots_ab'

# Create a list of filenames of saved plots
def extract_number(filename):
    match = re.search(r'\d+', filename)
    if match:
        return int(match.group())
    return -1

# Get a list of PNG files in the specified directory and sort them based on the numbers in their names
image_files = sorted([os.path.join(save_dir, f) for f in os.listdir(save_dir) if f.endswith('.png')], key=extract_number)

# Print sorted filenames along with the extracted numbers
for idx, file in enumerate(image_files, start=1):
    number = extract_number(file)
    print(f"{idx}. Number: {number}, File: {os.path.basename(file)}")

1. Number: 0, File: decision_boundary_epoch_0.png
2. Number: 1, File: decision_boundary_epoch_1.png
3. Number: 2, File: decision_boundary_epoch_2.png
4. Number: 3, File: decision_boundary_epoch_3.png
5. Number: 4, File: decision_boundary_epoch_4.png
6. Number: 5, File: decision_boundary_epoch_5.png
7. Number: 6, File: decision_boundary_epoch_6.png
8. Number: 7, File: decision_boundary_epoch_7.png
9. Number: 8, File: decision_boundary_epoch_8.png
10. Number: 9, File: decision_boundary_epoch_9.png
11. Number: 10, File: decision_boundary_epoch_10.png
12. Number: 11, File: decision_boundary_epoch_11.png
13. Number: 12, File: decision_boundary_epoch_12.png
14. Number: 13, File: decision_boundary_epoch_13.png
15. Number: 14, File: decision_boundary_epoch_14.png
16. Number: 15, File: decision_boundary_epoch_15.png
17. Number: 16, File: decision_boundary_epoch_16.png
18. Number: 17, File: decision_boundary_epoch_17.png
19. Number: 18, File: decision_boundary_epoch_18.png
20. Number: 19, File: 

In [13]:
# Create a list to store all frames
frames = []

# Load each image and append to frames list
for i, image_file in enumerate(image_files):
    img = Image.open(image_file)
    frames.append(img)
    # Print progress every 10 images
    if (i + 1) % 10 == 0 or (i + 1) == len(image_files):
        print(f'Loaded {i + 1}/{len(image_files)} images')
        
# Save as GIF
output_gif_path = 'ab.gif'
frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], duration=100, loop=0)

Loaded 10/1000 images
Loaded 20/1000 images
Loaded 30/1000 images
Loaded 40/1000 images
Loaded 50/1000 images
Loaded 60/1000 images
Loaded 70/1000 images
Loaded 80/1000 images
Loaded 90/1000 images
Loaded 100/1000 images
Loaded 110/1000 images
Loaded 120/1000 images
Loaded 130/1000 images
Loaded 140/1000 images
Loaded 150/1000 images
Loaded 160/1000 images
Loaded 170/1000 images
Loaded 180/1000 images
Loaded 190/1000 images
Loaded 200/1000 images
Loaded 210/1000 images
Loaded 220/1000 images
Loaded 230/1000 images
Loaded 240/1000 images
Loaded 250/1000 images
Loaded 260/1000 images
Loaded 270/1000 images
Loaded 280/1000 images
Loaded 290/1000 images
Loaded 300/1000 images
Loaded 310/1000 images
Loaded 320/1000 images
Loaded 330/1000 images
Loaded 340/1000 images
Loaded 350/1000 images
Loaded 360/1000 images
Loaded 370/1000 images
Loaded 380/1000 images
Loaded 390/1000 images
Loaded 400/1000 images
Loaded 410/1000 images
Loaded 420/1000 images
Loaded 430/1000 images
Loaded 440/1000 imag